In [5]:
import os
from xml2mask_v2 import xml2mask_v2
from matplotlib import pyplot as plt
import cv2
from time import time
import numpy as np
from PIL import Image
import pandas as pd
from copy import deepcopy

In [6]:
DL_src = r'\\kukissd\Kyu_Sync\deeplab with ashley\training_set_v2\5x\classification_for_DL_skin_v6_control'
DL_results = [os.path.join(DL_src,_) for _ in os.listdir(DL_src) if _.endswith('tif')]
# DL_results = [_ for _ in DL_results if not _.endswith(('024.tif','002.tif'))]

svs_src = r'\\kukissd\Kyu_Sync\Aging\data\svs'
svss = [os.path.join(svs_src,_) for _ in os.listdir(svs_src) if _.endswith('svs')]
# svss = [_ for _ in svss if not _.endswith(('024.svs','002.svs'))]
xml_src = r'\\kukissd\Kyu_Sync\deeplab with ashley\training_set_v2'
xmls = [os.path.join(xml_src,_) for _ in os.listdir(xml_src) if _.endswith('xml')]

rmwhite = True
if rmwhite: mask_src = r'\\kukissd\Kyu_Sync\deeplab with ashley\training_set_v2\data_DL_skin_v6_control'

In [7]:
sort_index = [11,0,1,10,9,3,2,4,5,6,7,8]
sort_index_new = [9,0,1,8,7,2,3,4,5,6]
accuracies = np.zeros((0,len(sort_index_new)+2))
# columns = ['filename','corneum','basale','root','follicle','muscle','oil','sweat','nerve','blood','collagen','fat','whitespace']
columns = ['filename','true_class','corneum','basale','muscle','oil','sweat','nerve','blood','collagen','fat','whitespace']
accuracies = pd.DataFrame(accuracies,columns=columns)

In [8]:
for idx,(svs_path,xml_path,DL) in enumerate(zip(svss,xmls,DL_results)):
    print('image progress',idx+1,'/',len(svss))
    if rmwhite:
        _,fn = os.path.split(svs_path)
        fn,_ = os.path.splitext(fn)
        mask_src = os.path.join(*[mask_src,fn,'view_annotations.tif'])
        mask = Image.open(mask_src)
    else:
        mask = xml2mask_v2(svs_path,xml_path,mag=5,sort_index=sort_index,binary=False)
        mask[mask==3]=1
        mask[mask==4]=2
        mask[mask==5]=3
        mask[mask==6]=4
        mask[mask==7]=5
        mask[mask==8]=6
        mask[mask==9]=7
        mask[mask==10]=8
        mask[mask==11]=9
        mask[mask==12]=10

    DL_result_img = np.array(Image.open(DL))
    mask = cv2.resize(mask, dsize=(DL_result_img.shape[::-1]), interpolation=cv2.INTER_NEAREST)
    # print(np.unique(mask))
    # iterate true class
    for classid in sorted(sort_index_new):
        print('class progress',classid+1,'/',len(sort_index_new))
        row=[]
        row.append(os.path.split(svs_path)[-1])
        row.append(columns[classid+2])
        classid = classid+1
        DL_result_img2 = deepcopy(DL_result_img)
        DL_result_img2[mask!=classid]=0
        # fig2,ax2=plt.subplots(1,2)
        # ax2[0].imshow((DL_result_img2==classid)*1)
        # ax2[1].imshow((mask==classid)*1)
        # iterate predict class
        for classidx in sorted(sort_index_new):
            a=(np.sum(np.sum(((DL_result_img2==classidx+1)*1),axis=1)/1)*1)
            b=(np.sum(np.sum(((mask==classid)*1),axis=1)/1)*1)
            # print(a,b)
            row.append(np.around(a/b,decimals=3))
        # accuracies.loc[idx] = row
        df2 = pd.DataFrame([row], columns=columns)
        accuracies = accuracies.append(df2, ignore_index=True)

    # fig2,ax2=plt.subplots(1,2)
    # fig2.set_figheight(12)
    # fig2.set_figwidth(24)
    # ax2[0].imshow((mask>0)*255,cmap='gray')
    # ax2[1].imshow(DL_result_img,cmap='gray')
    # ax2[1].set_title('accuracy = '+str(a/b))
    # fig2.savefig(os.path.join(DL_src,'svg/manual_vs_deeplab_collagen_within_manual_{:d}.svg'.format(idx)),format='svg')
    # plt.close(fig2)

accuracies.to_csv(os.path.join(DL_src,'multiclass_accuracies.csv'),index=False)

image progress 1 / 36
readxml took 0.38 sec
mask generation: 0.67 sec elapsed
class progress 1 / 10
class progress 2 / 10
class progress 3 / 10
class progress 4 / 10
class progress 5 / 10
class progress 6 / 10
class progress 7 / 10
class progress 8 / 10
class progress 9 / 10
class progress 10 / 10
image progress 2 / 36
readxml took 0.08 sec
mask generation: 1.60 sec elapsed
class progress 1 / 10
class progress 2 / 10
class progress 3 / 10
class progress 4 / 10
class progress 5 / 10
class progress 6 / 10
class progress 7 / 10
class progress 8 / 10
class progress 9 / 10
class progress 10 / 10
image progress 3 / 36
readxml took 0.05 sec
mask generation: 0.30 sec elapsed
class progress 1 / 10
class progress 2 / 10
class progress 3 / 10
class progress 4 / 10
class progress 5 / 10
class progress 6 / 10
class progress 7 / 10
class progress 8 / 10
class progress 9 / 10
class progress 10 / 10
image progress 4 / 36
readxml took 0.08 sec
mask generation: 0.29 sec elapsed
class progress 1 / 10
cla

C:\ProgramData\Anaconda3\envs\TF2Shared\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
